## Question 3:
(a) Create a parser to read class hierarchy files and a data structure to efficiently (minimise time complexity) perform the following operations.
- Find all siblings class of a class name
- Find the parent class of a class name
- Find all ancestor classes of a class name
- Find if both class 1 and class 2 belong to the same ancestor class(es)

Definition:

- Parent class: immediate superclass (up to 1 level in hierarchy)
- Ancestor class: all the superclasses (including parent of the parent class)
- Sibling class: classes that belong to the same parent class

You are provided with two files:

- `hierarchy.txt`: each line consists of a parent class and a child class
- `id_to_name.txt`: each line consists of a category id and its name
---
I solve this question by using **Disjoint set** with some customization.


In [ ]:
class DisjointSet:
	def __init__(self, n):
		# Ranking mechanism for boost performance of Disjoint set
		self.rank = [1] * n

		# "disjoint set" ancestor which used to check if two sets are disjoint or not.
		self.ancestor = [i for i in range(n)]

		# Store parent of node i
		self.parent = [i for i in range(n)]

		# Store children of node i
		self.childs = dict()


	# Finds set of given item x
	def find_ancestor(self, x):

		# Finds the representative of the set
		# that x is an element of
		if (self.ancestor[x] != x):

			# Path compression technique ~ caching
			self.ancestor[x] = self.find_ancestor(self.ancestor[x])

		return self.ancestor[x]

	# (***) Find all ancestor nodes of x
	# We can further improve this by using caching method
	def find_ancestors(self, x):
		ancestors = [self.parent[x]]
		crawl_node = self.parent[x]
		while(crawl_node != self.parent[crawl_node]):
			crawl_node = self.parent[crawl_node]
			ancestors.append(crawl_node)

		return ancestors

	# (***) Find all siblings nodes of x
	# Siblings of x = children of x's parent ecepting x
	def find_siblings(self, x):
		parent_x = self.find_parent(x)
		return set(self.childs[parent_x])  - set([x])

	# (***) Find the parent node of x
	def find_parent(self, x):
		return self.parent[x]

	def is_disjoint(self, x, y):
		return self.find_ancestor(x) != self.find_ancestor(y)

	# Do union of two sets represented
	# by x and y.
	def union(self, parent_node, child_node):
		# Set parent child relationship
		self.parent[child_node] = parent_node

		# Add this child to list of children of parent_node
		if parent_node not in self.childs:
			self.childs[parent_node] = [child_node]
		else:
			self.childs[parent_node].append(child_node)

		# Find current sets of x and y
		parent_set = self.find_ancestor(parent_node)
		child_set = self.find_ancestor(child_node)

		# If they are already in same set
		if parent_set == child_set:
			return

		if self.rank[parent_set] < self.rank[child_set]:
			self.ancestor[parent_set] = child_set

		elif self.rank[parent_set] > self.rank[child_set]:
			self.ancestor[child_set] = parent_set

		# If ranks are same, then move y under
		# x (doesn't matter which one goes where)
		# and increment rank of x's tree
		else:
			self.ancestor[child_set] = parent_set
			self.rank[child_set] = self.rank[parent_set] + 1


In [ ]:
class Parser:
    def __init__(self, hierachy_path, id_to_name_path):
        # ==== PATH ====
        self.hierachy_path = hierachy_path
        self.id_to_name_path = id_to_name_path

        # Id (of class) <-> Name (of class)
        self.id2class = dict()
        self.class2id = dict()

        # Id (of class) <-> Node (in disjoint set)
        self.id2node = dict()
        self.node2id = dict()

        # ----------------------
        self.__read_file__()

    def __read_file__(self):
        with open(self.id_to_name_path, 'r') as file:
            lines = file.readlines()
            self.n = len(lines)

            for index, line in enumerate(lines):
                # print(line)
                id, class_name = line.split("\t")
                id = id.strip()
                class_name = class_name.strip()
                # print(id, class_name)
                self.id2class[id] = class_name
                self.class2id[class_name] = id

                self.id2node[id] = index
                self.node2id[index] = id
        self.__build_djset__()

    def __build_djset__(self):
        # Create disjoint set
        self.disjoint_set = DisjointSet(self.n)

        with open(self.hierachy_path, "r") as file:
            lines = file.readlines()

            for line in lines:
                parent, child = line.split()
                parent = parent.strip()
                child = child.strip()
                parent_node, child_node = self.id2node[parent], self.id2node[child]

                # Add to disjoint set
                self.disjoint_set.union(parent_node=parent_node, child_node=child_node)

    def __name2node(self, class_name):
        return self.id2node[self.class2id[class_name]]
    def __node2name(self, node):
        return self.id2class[self.node2id[node]]

    def node2name(self, node):
        return self.__node2name(node)
    # (***) Find all siblings class of a class name
    def find_siblings(self, class_name):
        class_node = self.__name2node(class_name)
        siblings = self.disjoint_set.find_siblings(class_node)
        return [(self.node2id[x], self.__node2name(x)) for x in siblings]

    # (***) Find the parent class of a class name
    def find_parent(self, class_name):
        class_node = self.__name2node(class_name)
        parent_node = self.disjoint_set.find_parent(class_node)
        return (self.node2id[parent_node], self.__node2name(parent_node))

    # (***) Find all ancestor classes of a class name
    def find_ancestors(self, class_name):
        class_node = self.__name2node(class_name)
        ancestors = self.disjoint_set.find_ancestors(class_node)
        return [(self.node2id[x], self.__node2name(x)) for x in ancestors]

    # (***) Find all common ancestor classes of both class 1 and class 2
    def same_ancestor(self, class_name_1, class_name_2):
        class_node_1 = self.__name2node(class_name_1)
        class_node_2 = self.__name2node(class_name_2)

        same_ancestor = not self.disjoint_set.is_disjoint(class_node_1, class_node_2)

        if not same_ancestor:
            return set()

        ancestors_1 = set(self.find_ancestors(class_name_1))
        ancestors_2 = set(self.find_ancestors(class_name_2))

        intersect_set = ancestors_1.intersection(ancestors_2)
        return intersect_set

In [ ]:
parser = Parser("hierarchy.txt", "id_to_name.txt")
# Example for each required operation:
class_name = "cell"

In [ ]:
## Find all siblings class of a class name
print(f"Siblings class of [{class_name}]: {parser.find_siblings(class_name)}")

Siblings class of [cell]: [('n03144486', 'cubbyhole, pigeonhole'), ('n03990210', 'Post-Office box, PO Box, POB, call box, letter box')]


In [ ]:
## Find the parent class of a class name
print(f"Parent class of [{class_name}]: {parser.find_parent(class_name)}")

Parent class of [cell]: ('n03080309', 'compartment')


In [ ]:
## Find all ancestor classes of a class name
print(f"All ancestors class(es) of [{class_name}]:")
parser.find_ancestors(class_name)

All ancestors class(es) of [cell]:


[('n03080309', 'compartment'),
 ('n13910384', 'space'),
 ('n13867492', 'amorphous shape'),
 ('n00027807', 'shape, form'),
 ('n00024264', 'attribute'),
 ('n00002137', 'abstraction, abstract entity'),
 ('n00001740', 'entity')]

In [ ]:
## Find if both class 1 and class 2 belong to the same ancestor class(es)
class_name_1 = "gym suit"
class_name_2 = "cell"
# class_name_2 = "running suit"
print(f"Class [{class_name_1}] and class [{class_name_2}] belong to the same ancestor class(es):")
parser.same_ancestor(class_name_1, class_name_2)

Class [gym suit] and class [cell] belong to the same ancestor class(es):


{('n00001740', 'entity')}